In [2]:
import numpy as np
import scipy
class Sampling_Random_State:
    #### --------- Definition of variables ------------------------
    N_size=500001
    Gamma=0.5
    Lambda=0.5
    num_data = 2000
    #### ------------------------------------------------------------


    @classmethod
    def Alpha(cls,theta:np.float64)-> np.float64:
        return cls.Lambda+np.cos(theta)
    @classmethod
    def Beta(cls,theta:np.float64)-> np.float64:
        return cls.Gamma*np.sin(theta)
    @classmethod
    def Omega(cls,theta:np.float64)-> np.float64:
        return np.sqrt(cls.Alpha(theta)**2 + cls.Beta(theta)**2 )
    @classmethod
    def Phi(cls,theta:np.float64)-> np.float64:
        return np.arctan2(cls.Beta(theta),cls.Alpha(theta))



    @classmethod
    def Fermi_dirac(cls,n:np.int64,mu:np.float64 =0) -> np.float64:
        # beta is the inverse thermic energy associated in the system (beta)
        # mu corresponds to the chemical potential
        # n is the position of the particle
        # f=np.exp(T*(Omega(Gamma,Lambda,2.0*(np.pi/N)*n)-mu)) +1
        # N corresponds to the size of the system
        beta = np.min(cls.Omega(np.linspace(-np.pi,np.pi,int(1000))))
        f=np.exp(beta*(cls.Omega(((2.*np.pi)/np.float64(cls.N_size)) * n)-mu)) +1
        return 1/f

    @classmethod
    def Sample_State(cls,Ground:bool =False,mu:np.float64 = 0.0)-> np.ndarray:
        x=np.arange(0,(cls.N_size-1)/2+ 1)
        if Ground:
            m_cos=[-0.5 for i in x]
            m_sin=[-0.5 for i in x]
        else:
            m_cos=[-0.5 if np.random.random()>cls.Fermi_dirac(mu=mu,n=i) else 0.5 for i in x]
            m_sin=[-0.5 if np.random.random()>cls.Fermi_dirac(mu=mu,n=i) else 0.5 for i in x]
        x=np.arange(-(cls.N_size-1)/2,(cls.N_size-1)/2+1)
        M_minous=[((m_cos[np.abs(int(i))]-m_sin[np.abs(int(i))])*0.5*np.exp(1.j*np.sign((2.0*np.pi/cls.N_size) * i)*cls.Phi(np.abs((2.0*np.pi/cls.N_size) * i)))) for i in x]
        M_plus = [((m_cos[np.abs(int(i))]+m_sin[np.abs(int(i))])*0.5*np.exp(1.j*np.sign((2.0*np.pi/cls.N_size) * i)*cls.Phi(np.abs((2.0*np.pi/cls.N_size) * i)))) for i in x]
        Mminousband=np.array(M_minous)
        Mplusband=np.array(M_plus)
        return Mminousband,Mplusband

In [3]:
a=Sampling_Random_State()
M_m, M_p = a.Sample_State()

In [4]:
%%timeit
M_m, M_p = a.Sample_State()

5.35 s ± 163 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
np.fft.fft(np.roll(M_p,-(a.N_size-1)//2))

83.2 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
fft(ifftshift(M_p))

In [10]:
%%timeit
fft(np.roll(M_p,-(a.N_size-1)//2))

63.4 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit
np.fft.fft(np.fft.ifftshift(M_p))

87.3 ms ± 6.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
fft(ifftshift(M_p))

array([-2.48928917e+04-2.37395722e-15j, -3.81619661e+04-5.39124551e-13j,
        6.54036924e+02+8.28519179e-13j, ...,
       -1.28741304e+01-4.97809182e-15j, -3.83518975e+02-4.31023688e-14j,
       -1.20962596e+04-7.47043723e-13j])

In [4]:
np.fft.fft(np.fft.ifftshift(M_p))

array([-2.48928917e+04-1.03739239e-12j, -3.81619661e+04-1.40232270e-12j,
        6.54036924e+02-1.68358383e-13j, ...,
       -1.28741304e+01+8.32134600e-13j, -3.83518975e+02-7.40293243e-14j,
       -1.20962596e+04-3.59032248e-13j])

In [4]:
from pyfftw.interfaces import scipy_fftpack

In [5]:
%%timeit
scipy_fftpack.fft(np.fft.ifftshift(M_p))

62.5 ms ± 998 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
import pyfftw

In [17]:
%%timeit 
cosa = pyfftw.empty_aligned(a.N_size, dtype='complex128')
fft = pyfftw.builders.fft(cosa)
cosa[:] = np.fft.ifftshift(M_p)
fft() 

50.9 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
fft()

array([238733.89209364+1.60017555e-11j, -37136.96096592-5.30504871e+04j,
        -7831.79538166-2.12202000e+04j, ...,
        -3386.91153668+1.36415577e+04j,  -7831.79538166+2.12202000e+04j,
       -37136.96096592+5.30504871e+04j])

In [19]:
scipy_fftpack.fft(np.fft.ifftshift(M_p))

array([-25212.88579807-1.63424829e-12j, -37893.93624878-2.70297673e-12j,
          327.07879383+9.59267388e-14j, ...,
          276.08458093-1.58936745e-14j,    514.31141796-1.37255486e-13j,
       -12447.58796894-6.69353462e-13j])

In [20]:
np.fft.fft(np.fft.ifftshift(M_p))

array([-25212.88579807-9.05941988e-13j, -37893.93624878-1.45786161e-12j,
          327.07879383-4.66033462e-14j, ...,
          276.08458093+1.01208625e-12j,    514.31141796+6.73419653e-14j,
       -12447.58796894-4.09977607e-13j])

In [23]:
np.allclose(scipy_fftpack.fft(np.fft.ifftshift(M_p)),np.fft.fft(np.fft.ifftshift(M_p)))

True

In [30]:
numpy_arr = pyfftw.empty_aligned(a.N_size, dtype='complex128')

In [31]:
numpy_arr[:]=np.fft.ifftshift(M_p)
pyfftw.interfaces.numpy_fft.fft(numpy_arr)

array([-25212.88579807-1.63424829e-12j, -37893.93624878-2.70297673e-12j,
          327.07879383+9.59267388e-14j, ...,
          276.08458093-1.58936745e-14j,    514.31141796-1.37255486e-13j,
       -12447.58796894-6.69353462e-13j])

In [28]:
%%timeit
pyfftw.interfaces.numpy_fft.fft(numpy_arr)

51.6 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
numpy_arr

array([-0.5+0.00000000e+00j, -0.5-2.09439091e-06j,  0. +0.00000000e+00j,
       ...,  0. +0.00000000e+00j,  0. +0.00000000e+00j,
       -0.5+2.09439091e-06j])

In [20]:
cosa = pyfftw.empty_aligned(a.N_size, dtype='complex128')
fft = pyfftw.builders.fft(cosa)
cosa[:] = np.fft.ifftshift(M_p)

In [21]:
%timeit fft() 

21.1 ms ± 916 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%timeit scipy_fftpack.fft(np.fft.ifftshift(M_p))

55.6 ms ± 983 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
cosa

array([-0.5+0.00000000e+00j,  0. +0.00000000e+00j, -0.5-4.18878183e-06j,
       ...,  0. +0.00000000e+00j, -0.5+4.18878183e-06j,
        0. +0.00000000e+00j])